In [1]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict

# Avslutte manuelt innlagt i eldre emne

## Problemstilling
Når vi legg inn brukarar manuelt i emne vert dei liggande aktive etter at emneet er avslutta, for det vert ikkje sendt melding om å avslutte innmeldinga for desse, slik det vert gjort for dei som kjem frå FS.

## Løysing
Eg reknar alle emne som har `enrollment_term_id` i lista over `avslutt_semestre` som avslutta. Det er desse som får "avslutt"-melding frå FS våren 2025 (kanskje). 
Så går eg gjennom denne lista (ca. 16000 emne) og finn alle personar som framleis er aktive. 

## Løysing 1: Hente alle emne
Først hentar eg alle emna vi har i Canvas.

In [ ]:
url = "https://hvl.instructure.com/api/v1/accounts/1/courses"

dr_liste = []
hentmeir = True
while hentmeir:
    respons = requests.get(url, headers=headers, params=params)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
            print(url)   # for kontrollen sin skuld
        dataliste = []
        for element in data:
            dataliste.append([element['id'], element['name'], element['enrollment_term_id']])
        df = pd.DataFrame(dataliste, columns=['id', 'name', 'enrollment_term_id'])
        dr_liste.append(df)
# og etter at eg er ferdig å lese inn slår eg dei same:
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)


In [3]:
alledata.to_csv("alle_emne_250407.csv", index=False)

### Liste alle "eldre" emne

No kan eg plukke ut alle som har `enrollment_term_id` i `avslutt_semestre` og legge dei i ei eiga fil. Lista over terminnummer har eg henta i `terminar.ipynb`:

In [4]:
avslutt_semestre = [2,  6,  7,  8,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29, 30,  31,  32,  33,  34,  35,  36,  39,  40,  41,  73,  74,  75,  76,  77,  79,  81,  82,  83,  84,  85,  86,  87,  88,  90,  91,  92,  93,  196,  197,  198,  199,  200,  202,  203,  204,  205,  208,  209,  210,  212,  215,  220,  11] 
alledata[alledata['enrollment_term_id'].isin(avslutt_semestre)].to_csv("alle_emne_som _er _avslutta_250407.csv", index=False)

### Endre status

Dette bør eg kanskje ikkje gjere "sånn med ein gang" - eg bør varsle alle tilsette om at emne dei har hatt tilgang til no vert avslutta, og dermed flytta til filarkivet?

Men det er ikkje så viktig; dei aller, aller fleste tilsette er no vande med at emne dei har hatt før skal bli avslutta. Og eg ser berre på emne som har SIS-ID som starter på "UE" eller "UA".

Så eg kan gjere følgjande:

1. Gå gjennom alle emne i lista over "forelda"
2. Sjekk om emne har SIS-ID som starter på "UE" eller "UA"
3. Hvis ja, set alle påmeldingar til "concluded"



In [ ]:
emne = pd.read_csv("alle_emne_som _er _avslutta_250407.csv")
emneliste = []
params = {'per_page': 100}
for i, row in emne.iterrows():
    id = int(row['id'])
    url = f"https://hvl.instructure.com/api/v1/courses/{id}"
    resultat = requests.get(url, headers=headers, params=params)
    print(id, resultat.status_code)
    data = resultat.json()
    if 'sis_course_id' in data and data['sis_course_id'] is not None and data['sis_course_id'].startswith(("UA", "UE")):
        print(data['sis_course_id'])
        emneliste.append([data['id'], data['workflow_state'], data['sis_course_id']])

In [ ]:
len(emne) - len(emneliste)

## Testkanin
Eg prøver eit tilfeldig valgt emne frå våren 2018; der er det ein lærar inne (Dag Digernes) men ein haug med studentar. Eg set emnet inaktivt, og ser kva konsekvensar det får for læraren. Ein mistanke eg har er at han ikkje lengre kan kopiere frå emnet, men den viser seg å vere ugrunna. Det einaste som skjer er at han mister emnet frå dashbordet (dersom han hadde det der).

In [ ]:
emnekode = 228
url = f"https://hvl.instructure.com/api/v1/courses/{emnekode}"
params = {'course[event]': 'conclude'}
respons = requests.put(url, headers=headers, params=params)
respons.json()

## Vidare arbeid
Eg lagrar lista over emna i ei fil, og så ser eg kva vi gjer med  desse etter å ha diskutert med dei andre.

In [ ]:
df = pd.DataFrame(emneliste, columns=["id", "status", "sis_course_id"])
df.to_csv("emneliste.csv", index=False)
df['status'].value_counts()

Det er altså 8948 emne som er publisert. Kor mange av desse som har lærarar/studentar som er lagt innn manuelt er vanskeleg å sei utan å gå gjennom alle. Men vi har jo grunnar til å setje status til "concluded" for desse.